# Exploratory Data Analysis (EDA)

This script performs data checks, descriptive tables and the core visualizations required to understand the monthly Pulmonary Tuberculosis series (2001–2010) and to prepare inputs for time-series modeling and forecasting (2011 prediction).

## Step #1: Initiate environment for **Exploratory Data Analysis**
This section imports necessary libraries, creates debugging flags and control variables and configures paths.

In [ ]:
# EN: Setup cell - imports, debug/lang flags, paths, and plotting defaults
# PT-BR: Célula de setup - imports, flags de debug/idioma, caminhos e defaults de plot

DEBUG = True        # EN: Show debug messages?  PT-BR: Mostrar mensagens de debug?
LANGUAGE = "EN"     # EN: "EN" or "PT"  PT-BR: "EN" ou "PT"

import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import locale
import seaborn as sns
import itertools
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Plot defaults
plt.rcParams.update({"figure.figsize": (12, 5), "font.size": 12})
sns.set_style("whitegrid")

# Paths (adjust if needed)
DATA_DIR = Path("processed_data")
FIG_DIR = Path("figures")
TAB_DIR = Path("tables")
FIG_DIR.mkdir(exist_ok=True, parents=True)
TAB_DIR.mkdir(exist_ok=True, parents=True)

def debug(en_msg, pt_msg=""):
    if not DEBUG:
        return
    msg = en_msg if LANGUAGE.upper()=="EN" else (pt_msg if pt_msg else en_msg)
    print("[DEBUG]", msg)

debug("Setup completed. Data dir = processed_data/", "Setup completo. Pasta de dados = processed_data/")


### Step 2: **Load Data**
Load the preprocessed data for EDA.

In [ ]:
# EN: Load core CSVs (already preprocessed)
# PT-BR: Carregar CSVs principais (já pré-processados)

fn_filtered = DATA_DIR / "filtered_pulmonary_tb_rows.csv"
fn_monthly = DATA_DIR / "monthly_pulmonary_tb_by_month.csv"
fn_annual = DATA_DIR / "annual_pulmonary_tb_by_year.csv"

for f in (fn_filtered, fn_monthly, fn_annual):
    if not f.exists():
        raise FileNotFoundError(f"Required file not found: {f} — check pre-processing step. / Arquivo necessário não encontrado: {f} — verifique pré-processamento.")

df_raw = pd.read_csv(fn_filtered, low_memory=False)
df_monthly = pd.read_csv(fn_monthly, parse_dates=["date"] if "date" in pd.read_csv(fn_monthly, nrows=0).columns else None)
df_annual = pd.read_csv(fn_annual)

debug(f"Loaded: filtered={len(df_raw)} rows, monthly={len(df_monthly)} rows, annual={len(df_annual)} rows",
    f"Carregados: filtrados={len(df_raw)} linhas, mensal={len(df_monthly)} linhas, anual={len(df_annual)} linhas")

# Quick peek
df_raw.head()


### Step 3: **Check Data**
Check the loaded data for errors and inconsistencies.

In [ ]:
# EN: Quick sanity checks: dtypes, missing counts, date parse validation
# PT-BR: Checagens rápidas: dtypes, contagem de missings, validação de data

def print_section(title_en, title_pt):
    if LANGUAGE.upper()=="EN":
        print(title_en)
    else:
        print(title_pt)

print_section("=== RAW ROWS: info() ===", "=== LINHAS FILTRADAS: info() ===")
display(df_raw.info())

print_section("\n=== MONTHLY: head() & dtypes ===", "\n=== MENSAL: head() & tipos ===")
display(df_monthly.head())
display(df_monthly.dtypes)

print_section("\n=== Missing values (raw) ===", "\n=== Valores faltantes (raw) ===")
display(df_raw.isna().sum().to_frame("missing_count").sort_values("missing_count", ascending=False).head(40))

print_section("\n=== Missing values (monthly) ===", "\n=== Valores faltantes (mensal) ===")
display(df_monthly.isna().sum().to_frame("missing_count").sort_values("missing_count", ascending=False))


### Step 4: **Check Data Types and Statistics**
Check the loaded data for incompatible data types and extract basic statistics for initial investigation.

In [ ]:
# EN: Ensure 'date' is datetime and set monthly index; compute basic descriptive stats for monthly series
# PT-BR: Garantir que 'date' é datetime e definir index mensal; calcular estatísticas descritivas básicas

# Ensure date column:
if "date" in df_monthly.columns:
    df_monthly["date"] = pd.to_datetime(df_monthly["date"], errors="coerce")
else:
    # fallback: construct date if year/month columns exist
    if "year" in df_monthly.columns and "month" in df_monthly.columns:
        df_monthly["date"] = pd.to_datetime(df_monthly["year"].astype(str) + "-" + df_monthly["month"].astype(str) + "-01", errors="coerce")
    else:
        raise SystemExit("Monthly CSV lacks 'date' and no year/month columns — cannot proceed. / CSV mensal não tem 'date' nem year/month — não é possível prosseguir.")

# Set index
df_monthly = df_monthly.sort_values("date").set_index("date")
series = df_monthly["cases"].astype(float)

# Basic stats
stats = series.describe().to_frame().T
stats["cv"] = stats["std"] / stats["mean"]
stats = stats[["count","mean","std","min","25%","50%","75%","max","cv"]]
stats = stats.rename(columns={"50%":"median"})
print_section("Monthly series descriptive statistics:", "Estatísticas descritivas da série mensal:")
display(stats.T)

# Save table
stats.to_csv(TAB_DIR / "monthly_series_descriptive_stats.csv", index=False)
debug("Saved monthly descriptive stats table.", "Estatísticas mensais salvas.")

### Step 5: **Data visualization**

Calculate and plot the **time series, from 2001-jan to 2010-dec, for cases/month** data visualization.

Related questions:
- What is the overall **behavior** of the data over time?
- Is there any obvious **trend** (growth/decline) or **seasonality** visible?
- Where are the most apparent **outliers**?

In [ ]:
# EN: Plot monthly time series (line) and save figure
# PT-BR: Plot da série mensal (linha) e salvar figura

locale.setlocale(locale.LC_TIME, "en_US.UTF-8")

fig, ax = plt.subplots(figsize=(25,6))

# --- 1. Plotting and Axis Limits ---
ax.plot(series.index, series.values, marker='o', linestyle='-', markersize=4)

ax.set_title("Monthly Pulmonary Tuberculosis cases (time series)")
# ax.set_xlabel is omitted since custom year labels are created below.
ax.set_ylabel("Cases")

# Sets X-axis limits with a small padding (15 days) before the first and after the last data point.
data_inicio_com_margem = series.index[0] - timedelta(days=15)
data_fim_com_margem = series.index[-1] + timedelta(days=15)
ax.set_xlim(data_inicio_com_margem, data_fim_com_margem)

# --- 2. Month Labels (Major Ticks) ---
# Locates a major tick at the start of every month.
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
# Formats the major tick to show the abbreviated month name (Jan, Feb...).
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
# Rotates the month labels for better readability.
plt.xticks(rotation=90, fontsize=10)


# --- 3. Year Demarcation (Thick Grid Lines) ---
years = np.unique(series.index.year)
# Loop through each year to draw the vertical separation lines.
for y in years:
    start_of_year = datetime(y, 1, 1)
    # Draws a thick vertical line at the beginning of each year.
    ax.axvline(start_of_year, color='black', linewidth=2, zorder=0)


# --- 4. Centered Year Labels ---
ymin, ymax = ax.get_ylim()
# Defines the vertical position for the year label (7% below the minimum Y-axis value).
y_text = ymin - (ymax - ymin) * 0.07

for y in years:
    start = datetime(y, 1, 1)
    # Defines the end of the current year block (start of the next year or end of series).
    if y == years[-1]:
        # For the last year, define the end based on the last data point plus padding.
        end = series.index[-1] + timedelta(days=16)
    else:
        end = datetime(y+1, 1, 1)
        
    # Calculates the horizontal center point of the year block.
    center = start + (end - start) / 2
    
    # Places the year label centered horizontally in the block.
    # The "\n" adds a line break below the month labels.
    ax.text(center, y_text, str("\n" + str(y)), ha='center', va='top', fontsize=12)

# --- 5. Grid and Final Layout Adjustments ---
# Draws the general grid lines (using major ticks as reference).
ax.grid(True, which='major', linestyle='-', linewidth=0.5)

# Draws thinner grid lines for all years (this is a redundant step as axvline covers it).
for y in years:
    start_of_year = datetime(y, 1, 1)
    # Draws a simple vertical line for general grid (optional, as thick lines are already in place).
    ax.axvline(start_of_year, color='grey', linewidth=0.5)

# Adjusts the bottom margin to accommodate the rotated month labels and the centered year labels.
plt.subplots_adjust(bottom=0.18)

# --- 6. Saving and Display ---
fig_path = FIG_DIR / "ts_monthly_cases_line_excel_style.png"
fig.savefig(fig_path, bbox_inches="tight", dpi=150)
print(f"Saved time series plot to {fig_path}")

plt.show()

Generate **boxplots**, from jan to dec yearly, for **seasonality investigation**.

Related questions:
- Does the series have a recurring **seasonal pattern**?
- What is the **dispersion and median** of cases in each month of the year?
- Where are the largest **outliers** relative to each seasonal period?

In [ ]:
# EN: Boxplot by month to spot seasonality (month-of-year)
# PT-BR: Boxplot por mês para identificar sazonalidade (mês do ano)

df_monthly_plot = df_monthly.reset_index()

# Extracts the numeric month (1 to 12) from the date column for plotting purposes.
df_monthly_plot["month_num"] = df_monthly_plot["date"].dt.month
# Defines the explicit order for the months (1 through 12) to ensure chronological display.
month_order = list(range(1,13))

# Generates the boxplot: X-axis is the month number, Y-axis is the case count.
plt.figure(figsize=(12,6))
sns.boxplot(x="month_num", y="cases", data=df_monthly_plot, order=month_order)
plt.title("Monthly distribution of cases by month-of-year")
plt.xlabel("Month")
plt.ylabel("Cases")
plt.xticks(ticks=np.arange(12), labels=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"])

# Saves the figure to the specified file path.
box_path = FIG_DIR / "boxplot_month_by_month.png"
plt.savefig(box_path, bbox_inches="tight", dpi=150)
debug(f"Saved month-wise boxplot to {box_path}", f"Boxplot mensal salvo em {box_path}")
plt.show()

Generate **heatmaps** (month x year), from 2001-jan to 2010-dec, for **pattern visualization**.

Related questions:
- How is the series' **value distributed** across the Month vs. Year axes?
- Is the **seasonality consistent** over the years, or has it intensified/softened over time?

In [ ]:
# EN: Heatmap (years x months) to visualize patterns across years and months
# PT-BR: Heatmap (anos x meses) para visualizar padrões ao longo dos anos e meses

# --- 1. Data Preparation (Pivot Table) ---
# Creates a pivot table to structure the data for the heatmap.
# Index (rows) are the years, columns are the months, and values are the total 'cases'.
pivot = df_monthly_plot.pivot_table(index=df_monthly_plot["date"].dt.year, 
                                    columns=df_monthly_plot["date"].dt.month, 
                                    values="cases", 
                                    aggfunc="sum")
# Reorders the columns to ensure they are sorted chronologically (Jan to Dec).
pivot = pivot.reindex(index=sorted(pivot.index), columns=month_order)

# --- 2. Heatmap Generation ---
plt.figure(figsize=(12,6))
# Generates the heatmap using the pivot table.
sns.heatmap(pivot, 
            annot=True,              # Annotate each cell with its value.
            fmt=".0f",               # Format the annotation as integers (no decimals).
            cmap="YlOrRd",           # Sets the color map (Yellow-Orange-Red, suitable for intensity).
            cbar_kws={'label':'cases'}) # Labels the color bar.
            
# Plot
plt.title("Heatmap: Year x Month cases")
plt.xlabel("Month")
plt.ylabel("Year")
plt.xticks(ticks=np.arange(12), labels=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"])

# --- 3. Saving and Output ---
heat_path = FIG_DIR / "heatmap_year_month_cases.png"
plt.savefig(heat_path, bbox_inches="tight", dpi=150)
debug(f"Saved heatmap to {heat_path}", f"Heatmap salvo em {heat_path}")
plt.show()
pivot.to_csv(TAB_DIR / "pivot_year_month_cases.csv")
debug("Saved pivot table year-month to tables.", "Pivot ano-mes salvo em tables.")

Calculate and **plot the histogram for frequency distribution** visualization.

Related questions:
- What is the **frequency distribution** of the values in the series?
- Does the data follow a **normal distribution** (bell curve)?
- What is the **smooth shape** of the frequency distribution?
- What is the most likely value (**mode**) of the data?

In [ ]:
# EN: Histogram and KDE for monthly counts
# PT-BR: Histograma e KDE para contagem mensal

# Generates the histogram plot.
# 'series.dropna()' ensures only valid case counts are used.
# 'kde=True' adds the Kernel Density Estimate line to show the smooth distribution shape.
# 'bins=30' specifies that the data range should be divided into 30 intervals (bars).
plt.figure(figsize=(10,5))
sns.histplot(series.dropna(), kde=True, bins=30)
plt.title("Histogram of monthly cases")
plt.xlabel("Cases")

# Saves the figure. 'bbox_inches="tight"' prevents labels from being cut off.
hist_path = FIG_DIR / "histogram_monthly_cases.png"
plt.savefig(hist_path, bbox_inches="tight", dpi=150)
debug(f"Saved histogram to {hist_path}", f"Histograma salvo em {hist_path}")
plt.show()

### Step 6: **Data Analysis**

**Decomposes the time series** to analyze the data's fundamental structure and **verify the presence of trend and seasonal patterns**.

Related questions:
- What is the **underlying level** of the series, excluding fluctuations (Trend)?
- What is the exact, **repetitive variation** that occurs in each cycle (Seasonality)?
- What **remains** in the series after removing the Trend and Seasonality (Residual)?

In [ ]:
# EN: STL decomposition (trend, seasonal, resid). Period=12 for monthly seasonality.
# PT-BR: Decomposição STL (tendência, sazonal, resíduo). Período=12 para sazonalidade mensal.

try:
    # --- Time Series Decomposition ---
    # Breaks down the time series into its core components (Trend, Seasonality, and Residual)
    # to understand the underlying structure of the data.
    decomposition = seasonal_decompose(series.dropna(), 
                                       model='additive',       # Assumes components are summed: Y = Trend + Seasonality + Residual.
                                       period=12,              # Sets the seasonal cycle length to 12 (as data is monthly, this represents the annual cycle).
                                       extrapolate_trend='freq') # Extrapolates the trend line to the edges of the series.
    
    # --- Plotting ---
    fig = decomposition.plot()
    fig.set_size_inches(12,8)
    plt.xlabel("\nYear")

    # --- Saving ---
    stl_path = FIG_DIR / "stl_decomposition.png"
    fig.savefig(stl_path, bbox_inches="tight", dpi=150)
    debug(f"Saved STL decomposition to {stl_path}", f"Decomposição STL salva em {stl_path}")
    plt.show()
# --- Error Handling ---
except Exception as e:
    # Analytical Note: The decomposition fails if there are not enough data points (less than two full cycles) 
    # or if the data still contains NaN values, leading to an exception.
    debug(f"STL decomposition failed: {e}", f"Decomposição STL falhou: {e}")
    print("STL decomposition failed — likely too few data points or NaNs. / Falha na decomposição STL — provavelmente poucos pontos ou NaNs.")

Calculates and **plots the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACP)** of the time series.

Related questions:
- Does the series have a **correlation with its own past** values?
- What is the **total reach** (direct and indirect) of this correlation?
- What is the direct, **pure correlation** between the current value and a specific lag, isolating the influence of intermediate lags?

In [ ]:
# EN: Plot ACF and PACF to inspect autocorrelation and help choose ARIMA orders
# PT-BR: Plot ACF e PACF para inspecionar autocorrelação e ajudar a escolher ordens ARIMA

# --- 1. Autocorrelation Function (ACF) Plot ---
plt.figure(figsize=(12,4))
# Generates the ACF plot: measures the total correlation between the series (Yt) and its past values (Yt-k).
# series.dropna(): Ensures clean data is used for correlation calculation.
# lags=36: Analyzes correlation up to 36 months (3 years) into the past.
# Purpose: The plot visually identifies the correlation structure of the series, indicating the order of the 
# Moving Average (MA) component, or 'q', in an ARIMA model (where the ACF cuts off or decays).
plot_acf(series.dropna(), lags=36)
plt.tight_layout()
plt.xlabel("Lag (months)")
acf_path = FIG_DIR / "acf_monthly.png"
plt.savefig(acf_path, bbox_inches="tight", dpi=150)
debug(f"Saved ACF plot to {acf_path}", f"ACF salvo em {acf_path}")
plt.show()

# --- 2. Partial Autocorrelation Function (PACF) Plot ---
plt.figure(figsize=(12,4))
# Generates the PACF plot: measures the pure, direct correlation between the series (Yt) and a specific lag (Yt-k), 
# after removing the influence of all intermediate lags (Yt-1, ..., Yt-k+1).
# lags=36: Analyzes direct correlation up to 36 months into the past.
# method='ywm': Uses the Yule-Walker method for PACF calculation, which is robust.
# Purpose: The plot is used to identify the order of the Autoregressive (AR) component, or 'p', in an ARIMA model 
# (where the PACF cuts off or decays).
plot_pacf(series.dropna(), lags=36, method='ywm')
plt.tight_layout()
plt.xlabel("Lag (months)")
pacf_path = FIG_DIR / "pacf_monthly.png"
plt.savefig(pacf_path, bbox_inches="tight", dpi=150)
debug(f"Saved PACF plot to {pacf_path}", f"PACF salvo em {pacf_path}")
plt.show()

**Aggregates the monthly data** into a yearly summary table to calculate the percentage change, allowing for analysis of long-term trends and growth/decline rates.

In [ ]:
# EN: Yearly aggregated table and year-on-year percent change
# PT-BR: Tabela agregada anual e variação percentual ano-a-ano

if "cases" in df_annual.columns:
    df_annual_sorted = df_annual.sort_values("year").copy()
else:
    # fallback: aggregate from monthly
    df_annual_sorted = df_monthly_plot.groupby(df_monthly_plot["date"].dt.year)["cases"].sum().reset_index().rename(columns={"date":"year"})

df_annual_sorted["pct_change"] = df_annual_sorted["cases"].pct_change() * 100
display(df_annual_sorted)
df_annual_sorted.to_csv(TAB_DIR / "annual_summary_cases_pctchange.csv", index=False)
debug("Saved annual summary table.", "Resumo anual salvo.")

### EDA Output

**Saves the EDA results** for later use.

In [ ]:
# EN: Save compact EDA summary for later use (quick tables)
# PT-BR: Salvar um resumo EDA compacto para uso posterior

# Monthly seasonal means
seasonal_means = df_monthly_plot.groupby(df_monthly_plot["date"].dt.month)["cases"].mean().rename_axis("month").reset_index()
seasonal_means["month_name"] = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
seasonal_means.to_csv(TAB_DIR / "monthly_seasonal_means.csv", index=False)

# Top months (highest counts)
top_months = df_monthly_plot.sort_values("cases", ascending=False).head(20)
top_months.to_csv(TAB_DIR / "top_20_months_by_cases.csv", index=False)

debug("Saved seasonal means and top months tables.", "Salvo médias sazonais e top meses.")

**List** the generated files.

In [ ]:
# EN: Optional - list generated files
# PT-BR: Opcional - listar arquivos gerados

print("Figures saved to:", FIG_DIR.resolve())
print("Tables saved to:", TAB_DIR.resolve())
print("Files in figures (example):", [p.name for p in sorted(FIG_DIR.glob("*.png"))][:10])
print("Files in tables (example):", [p.name for p in sorted(TAB_DIR.glob("*.csv"))][:20])

# Modeling and Forecast
Time series diagnostics, SARIMA/ETS model comparison, and forecast (train: 2001-2009, test: 2010, forecast: 2011). Builds candidate time-series models for the monthly pulmonary tuberculosis series, performs backtesting on 2010, compares metrics, and produces a final forecast for 2011 with prediction intervals.

### Step 1: **Stationarity Testing**
**Checks if the time series is stationary** by performing the ADF (Augmented Dickey-Fuller Test) to verify for a unit root, and the KPSS (Kwiatkowski-Phillips-Schmidt-Shin Test) to verify stationarity around a mean level or deterministic trend.

In [ ]:
# EN: Run ADF and KPSS tests to check stationarity
# PT-BR: Executa testes ADF e KPSS para verificar estacionariedade

# Define ADF test function
def adf_test(x):
    # adfuller: Performs the Augmented Dickey-Fuller test. 
    # autolag='AIC': Selects the optimal lag length based on the Akaike Information Criterion.
    r = adfuller(x.dropna(), autolag='AIC')
    # Returns the test statistics, p-value (key for decision), used lag, and number of observations.
    return {"adf_stat": r[0], "pvalue": r[1], "usedlag": r[2], "nobs": r[3]}

# Define KPSS test function
def kpss_test(x):
    # kpss: Performs the Kwiatkowski-Phillips-Schmidt-Shin test.
    # nlags="auto": Automatically determines the number of lags for the test.
    r = kpss(x.dropna(), nlags="auto")
    # Returns the test statistics, p-value, and number of lags used.
    return {"kpss_stat": r[0], "pvalue": r[1], "nlags": r[2]}

# Executes the tests on the series data.
adf_res = adf_test(series)
kpss_res = kpss_test(series)

debug(f"ADF: {adf_res}", f"ADF: {adf_res}")
debug(f"KPSS: {kpss_res}", f"KPSS: {kpss_res}")

# Prints the p-values for direct interpretation.
print("ADF p-value (stationary if < 0.05):", adf_res["pvalue"])
print("KPSS p-value (stationary if > 0.05):", kpss_res["pvalue"])

### Step 2: **Set Definition**
Defines the training set, for modeling, and the testing set, for forecasting performance evaluation.

In [ ]:
# EN: Train/test split: train <= 2009-12-01, test = 2010-01..2010-12
# PT-BR: Divisão Treino/Teste: treino <= 01/Dez/2009, teste = Jan/2010 até Dez/2010

# Define the end date for the training set (inclusive).
train_end = pd.to_datetime("2009-12-01")
# Define the start date for the test set.
test_start = pd.to_datetime("2010-01-01")
# Define the end date for the test set (inclusive).
test_end = pd.to_datetime("2010-12-01")

# Creates the training set: all observations up to and including the train_end date.
train = series[series.index <= train_end].copy()
# Creates the test set: all observations between test_start and test_end (inclusive).
test = series[(series.index >= test_start) & (series.index <= test_end)].copy()

# Prints the number of data points in each set for verification.
debug(f"Train points: {len(train)}, Test points: {len(test)}", f"Treino: {len(train)}, Teste: {len(test)}")
# Displays the last 6 observations of the training set to check the cutoff date.
display(train.tail(6))
# Displays the first 6 observations of the test set to check the starting date.
display(test.head(6))

### Step 3: **Evaluation Metrics**
Implements three metrics to quantify the forecasting models and regression error:
- MAE (Mean Absolute Error)
- RMSE (Root Mean Squared Error)
- MAPE (Mean Absolute Percentage Error)

In [ ]:
# EN: Define evaluation metrics
#PT-BR: Define métricas de avaliação

# --- 1. Mean Absolute Error (MAE) ---
def mae(y_true, y_pred):
    # Calculates the average of the absolute errors.
    return mean_absolute_error(y_true, y_pred)

# --- 2. Root Mean Squared Error (RMSE) ---
def rmse(y_true, y_pred):
    # Calculates the square root of the Mean Squared Error (MSE), returning the error to the original unit.
    return np.sqrt(mean_squared_error(y_true, y_pred))

# --- 3. Mean Absolute Percentage Error (MAPE) ---
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # CRITICAL FIX: Creates a mask to exclude zero values in y_true, preventing division by zero.
    mask = y_true != 0
    
    # Handles the edge case where all true values are zero.
    if mask.sum() == 0:
        return np.nan
        
    # Calculates the mean of the absolute percentage errors (only for non-zero true values), multiplied by 100.
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

### Step 4: **Modeling strategy and Training**
We'll fit two families of models:
- SARIMA via statsmodels (seasonal order with s=12). We'll perform a small grid search over (p,d,q) x (P,D,Q) limited ranges to keep runtime reasonable.
- ETS (Holt-Winters) via statsmodels' ExponentialSmoothing as a benchmark.
We will evaluate on the 2010 holdout and choose the best model by RMSE/MAE/MAPE.

#### SARIMA

**Hyperparameters optimization**

In [ ]:
# EN: SARIMA grid search (limited) — careful with runtime
# PT-BR: Busca em grade SARIMA (limitada) — atenção ao tempo de execução

# Sets random seed for reproducibility.
np.random.seed(42)

# Defines the search ranges for the non-seasonal (p, d, q) and seasonal (P, D, Q) orders.
ps = [0,1,2]
ds = [0,1]
qs = [0,1,2]
Ps = [0,1]
Ds = [0,1]
Qs = [0,1]
s = 12 # Sets the seasonal period to 12 (months).

best_sarima = None
results = []
start_time = time.time()

# --- Grid Search Loop ---
# We'll fit SARIMA on TRAIN and forecast the 12 months of TEST, compute RMSE
# Iterates through all combinations of non-seasonal parameters (p, d, q).
for p,d,q in itertools.product(ps,ds,qs):
    # Iterates through all combinations of seasonal parameters (P, D, Q).
    for P,D,Q in itertools.product(Ps,Ds,Qs):
        order = (p,d,q)
        seasonal_order = (P,D,Q,s)
        try:
            # Initializes the SARIMAX model using the current parameter combination.
            # train: The model is fitted only on the training data.
            # enforce_stationarity/invertibility=False: Allows the fitting of a wider range of models.
            model = SARIMAX(train, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
            # Fits the model (disp=False suppresses optimization output).
            res = model.fit(disp=False, method="lbfgs", maxiter=500)
            
            # --- Forecasting and Evaluation ---
            # Predicts the values for the test period (12 months).
            pred = res.get_forecast(steps=len(test))
            yhat = pred.predicted_mean
            
            # Computes evaluation metrics by comparing predictions (yhat) with real values (test).
            score_rmse = rmse(test.values, yhat.values)
            score_mae = mae(test.values, yhat.values)
            score_mape = mape(test.values, yhat.values)
            
            # Stores the parameters and scores.
            results.append({"order":order, "seasonal_order":seasonal_order, "rmse":score_rmse, "mae":score_mae, "mape":score_mape, "aic":res.aic})
            debug(f"SARIMA tried order={order} seasonal={seasonal_order} RMSE={score_rmse:.2f}", f"SARIMA testado order={order} seasonal={seasonal_order} RMSE={score_rmse:.2f}")
        
        except Exception as e:
            # Skips parameter combinations that fail to converge or result in errors.
            debug(f"SARIMA failed for order={order} seasonal={seasonal_order}: {e}", f"SARIMA falhou para order={order} seasonal={seasonal_order}: {e}")
            continue

# --- Results Processing ---
# Converts the results list into a DataFrame and sorts by RMSE to find the best model.
if results:
    # Creates and sorts DataFrame, but only if results is not empty.
    df_sarima_results = pd.DataFrame(results).sort_values("rmse").reset_index(drop=True)
else:
    # Handles the empty list case to avoid KeyError.
    print("No SARIMA models successfully fitted. Creating empty DataFrame.")
    df_sarima_results = pd.DataFrame() # Creates a safe empty DataFrame.

# Logs the completion time and the parameters of the best performing model.
debug(f"SARIMA grid search completed in {time.time()-start_time:.1f}s; best candidate: {df_sarima_results.iloc[0].to_dict() if not df_sarima_results.empty else 'none'}",
    f"Busca SARIMA concluída em {time.time()-start_time:.1f}s; melhor candidato: {df_sarima_results.iloc[0].to_dict() if not df_sarima_results.empty else 'nenhum'}")

# Displays the top 10 models ranked by RMSE.
display(df_sarima_results.head(10))

**Best model selection**

In [ ]:
# EN: Fit the best SARIMA from grid search on full TRAIN data (retrain) and produce forecast for 2011 (12 months)
# PT-BR: Ajustar o melhor SARIMA da busca em grade nos dados completos de TREINO (refit) e produzir previsão para 2011 (12 meses)

# Checks if the grid search returned any valid candidate models.
if df_sarima_results.empty:
    debug("No SARIMA candidate found — skipping SARIMA", "Nenhum candidato SARIMA encontrado — pulando SARIMA")
else:
    # --- 1. Model Selection ---
    # Retrieves the parameters of the best model (lowest RMSE) from the sorted DataFrame.
    best = df_sarima_results.iloc[0]
    best_order = tuple(best["order"])
    best_seasonal = tuple(best["seasonal_order"])
    debug(f"Refitting best SARIMA order={best_order} seasonal={best_seasonal}", f"Refit do melhor SARIMA order={best_order} seasonal={best_seasonal}")

    # --- 2. Final Retraining (Refit) ---
    # Initializes the SARIMAX model using the optimal parameters found.
    sarima_model = SARIMAX(train, order=best_order, seasonal_order=best_seasonal, enforce_stationarity=False, enforce_invertibility=False)
    # Fits (retrains) the model on the full training set. maxiter=500 increases optimization attempts.
    sarima_res = sarima_model.fit(disp=False, method="lbfgs", maxiter=500)
    
    # --- 3. Forecast for Test Period (Validation Plot) ---
    # Generates a forecast for the previously evaluated test period (2010).
    sarima_pred_test = sarima_res.get_forecast(steps=len(test))
    sarima_pred_mean_test = sarima_pred_test.predicted_mean
    # Calculates the 95% confidence interval (CI) for the test forecast.
    sarima_pred_ci_test = sarima_pred_test.conf_int(alpha=0.05)

    # --- 4. Final Forecast (Future Prediction) ---
    # Generates the final 12-month forecast for the future period (2011).
    sarima_forecast_2011 = sarima_res.get_forecast(steps=12)
    sarima_forecast_mean = sarima_forecast_2011.predicted_mean
    # Calculates the 95% confidence interval (CI) for the 2011 forecast.
    sarima_forecast_ci = sarima_forecast_2011.conf_int(alpha=0.05)

    # --- 5. Model Persistence ---
    # Saves the final fitted model object using pickle for later use (e.g., deployment or future predictions).
    import pickle
    with open(DATA_DIR / "sarima_model.pkl", "wb") as f:
        pickle.dump(sarima_res, f)

    debug("SARIMA model refit and saved to processed_data/sarima_model.pkl",
        "SARIMA refitado e salvo em processed_data/sarima_model.pkl")

#### (ETS/Holt-Winters)

**Model selection, training, forecasting, error handling and persistence**

In [ ]:
# EN: Fit ETS (Holt-Winters) on TRAIN and forecast
# PT-BR: Ajusta ETS (Holt-Winters) no conjunto de TREINO e gera previsão

# We'll try additive trend + additive seasonality as baseline; user can tune
try:
    # Initializes the Exponential Smoothing model (Holt-Winters).
    # trend="add" (additive trend): Assumes constant growth/decline.
    # seasonal="add" (additive seasonality): Assumes constant seasonal magnitude.
    # seasonal_periods=12: Sets the seasonal cycle length to 12 months.
    ets = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=12, initialization_method="estimated")
    
    # Fits the model by optimizing the smoothing parameters (alpha, beta, gamma).
    ets_res = ets.fit(optimized=True)
    
    # Generates a forecast for the test period (2010) for evaluation comparison.
    ets_pred_test = ets_res.forecast(steps=len(test))
    
    # Generates the final 12-month forecast for the future period (2011).
    ets_forecast_2011 = ets_res.forecast(steps=12) 
    
    # Save ETS model
    import pickle
    # Saves the fitted model object using pickle for later use/persistence.
    with open(DATA_DIR / "ets_model.pkl", "wb") as f:
        pickle.dump(ets_res, f)
        
    debug("ETS model trained and saved", "ETS treinado e salvo")
except Exception as e:
    # Handles potential convergence errors during the fitting process.
    debug(f"ETS fitting failed: {e}", f"Falha no ajuste ETS: {e}")
    ets_res = None

#### SARIMA and ETS evaluation

In [ ]:
# EN/PT: Evaluate SARIMA and ETS on the 2010 test set
# PT-BR: Avalia SARIMA e ETS no conjunto de teste de 2010

eval_rows = []
# --- Evaluation for SARIMA Model ---
if not df_sarima_results.empty:
    # Uses the mean forecast generated by the best SARIMA model for the test period.
    sarima_yhat = sarima_pred_mean_test
    eval_rows.append({
        "model":"SARIMA",
        # Calculates RMSE, MAE, and MAPE comparing the forecast (yhat) against the actual test values.
        "rmse": rmse(test.values, sarima_yhat.values),
        "mae": mae(test.values, sarima_yhat.values),
        "mape": mape(test.values, sarima_yhat.values)
    })

# --- Evaluation for ETS Model ---
if ets_res is not None:
    # Uses the forecast generated by the fitted ETS model for the test period.
    ets_yhat = ets_pred_test
    eval_rows.append({
        "model":"ETS",
        # Calculates RMSE, MAE, and MAPE for the ETS model.
        "rmse": rmse(test.values, ets_yhat.values),
        "mae": mae(test.values, ets_yhat.values),
        "mape": mape(test.values, ets_yhat.values)
    })

# --- Final Consolidation ---
# Converts the results list into a DataFrame.
df_eval = pd.DataFrame(eval_rows).sort_values("rmse").reset_index(drop=True)
# Displays the comparison table, with the best model (lowest RMSE) at the top.
display(df_eval)
# Saves the evaluation results to a CSV file.
df_eval.to_csv(DATA_DIR / "model_evaluation_on_2010.csv", index=False)
debug("Saved evaluation table to processed_data/model_evaluation_on_2010.csv",
    "Avaliação salva em processed_data/model_evaluation_on_2010.csv")

### Step 5: **Results presentation**

**Plots** the training data followed by testing data and the predictions of each model.

In [ ]:
# EN: Plot test period (2010) actual vs predictions (SARIMA and ETS)
# PT-BR: Plota período de teste (2010) observados vs predições (SARIMA e ETS)

plt.figure(figsize=(12,6))
# Plots the historical data (Train set) for context.
plt.plot(train.index, train, label="Train", color="black")
# Plots the actual observed values for the test period (2010).
plt.plot(test.index, test, label="Actual 2010", color="black", linestyle="--", marker='o')

# --- Plotting SARIMA Forecast ---
if not df_sarima_results.empty:
    # Plots the mean forecast generated by the best SARIMA model.
    plt.plot(test.index, sarima_pred_mean_test, label="SARIMA pred (2010)", marker='o')
    ci = sarima_pred_ci_test
    # Adds the 95% Confidence Interval (CI) as a shaded area around the forecast line.
    plt.fill_between(test.index, ci.iloc[:,0], ci.iloc[:,1], color='C0', alpha=0.2)

# --- Plotting ETS Forecast ---
if ets_res is not None:
    # Plots the forecast generated by the fitted ETS model.
    plt.plot(test.index, ets_pred_test, label="ETS pred (2010)", marker='o')

# Sets the title, displays the legend, and adds gridlines for readability.
plt.title("Model predictions vs actual (2010) / Predições vs observados (2010)")
plt.legend()
plt.grid(True)
plt_path = FIG_DIR / "model_predictions_vs_actual_2010.png"
# Saves the resulting visualization.
plt.savefig(plt_path, bbox_inches="tight", dpi=150)
debug(f"Saved prediction vs actual plot: {plt_path}", f"Plot predições vs observados salvo: {plt_path}")
plt.show()

Checks if the traning was successful and persists the forecasting in a DataFrame.

In [ ]:
# EN: Produce and save final 2011 forecasts (SARIMA and ETS) with intervals where available
# PT-BR: Produz e salva as previsões finais de 2011 (SARIMA e ETS) com intervalos onde disponíveis

forecasts = []

# --- 1. SARIMA Forecast Consolidation ---
if not df_sarima_results.empty:
    sarima_fc = sarima_forecast_mean
    sarima_ci = sarima_forecast_ci
    # Creates a DataFrame for the SARIMA forecast, including the 95% Confidence Interval (CI).
    sarima_df = pd.DataFrame({
        "date": pd.date_range(start="2011-01-01", periods=12, freq="MS"),
        "forecast_sarima": sarima_fc.values,
        "lower_sarima": sarima_ci.iloc[:,0].values,
        "upper_sarima": sarima_ci.iloc[:,1].values
    })
    forecasts.append(sarima_df)

# --- 2. ETS Forecast Consolidation ---
if ets_res is not None:
    ets_fc = ets_forecast_2011
    # Creates a DataFrame for the ETS forecast (point forecast only).
    ets_df = pd.DataFrame({
        "date": pd.date_range(start="2011-01-01", periods=12, freq="MS"),
        "forecast_ets": ets_fc.values
    })
    forecasts.append(ets_df)

# --- 3. Merging and Saving ---
# Merges all collected forecast DataFrames into a single comparison table.
if forecasts:
    df_forecast = forecasts[0]
    for f in forecasts[1:]:
        df_forecast = df_forecast.merge(f, on="date", how="outer")
        
    # Saves the final 2011 forecasts to CSV.
    df_forecast.to_csv(DATA_DIR / "forecast_2011_models.csv", index=False)
    debug("Saved 2011 forecasts to processed_data/forecast_2011_models.csv",
        "Previsões 2011 salvas em processed_data/forecast_2011_models.csv")
    display(df_forecast)
else:
    debug("No forecasts produced.", "Nenhuma previsão produzida.")

**The cherry of the cake**: plots the complete dataset, followed by its projection for 2011 with a shaded area for confidence interval.

In [ ]:
# EN: Plot history (2001-2010) + forecasts for 2011 with intervals
# PT-BR: Plota histórico (2001-2010) + previsões para 2011 com intervalos

plt.figure(figsize=(12,6))
# Plots the entire historical time series data (2001-2010).
plt.plot(series.index, series.values, label="History (2001-2010)", color="black")

# --- Plotting SARIMA Forecast for 2011 ---
if not df_sarima_results.empty:
    # Plots the SARIMA point forecast (mean) for the 12 months of 2011.
    plt.plot(df_forecast["date"], df_forecast["forecast_sarima"], label="SARIMA 2011", marker='o')
    # Adds the 95% Confidence Interval (CI) as a shaded area, representing prediction uncertainty.
    plt.fill_between(df_forecast["date"], df_forecast["lower_sarima"], df_forecast["upper_sarima"], color='C0', alpha=0.15)

# --- Plotting ETS Forecast for 2011 ---
if ets_res is not None:
    # Plots the ETS point forecast (mean) for the 12 months of 2011.
    plt.plot(df_forecast["date"], df_forecast["forecast_ets"], label="ETS 2011", marker='o')

# Sets the title, legend, and grid, finalizing the visualization.
plt.title("History + Forecast (2011) / Histórico + Previsão (2011)")
plt.legend()
plt.grid(True)
plt_path = FIG_DIR / "history_plus_forecast_2011.png"
# Saves the final figure.
plt.savefig(plt_path, bbox_inches="tight", dpi=150)
debug(f"Saved history + forecast plot: {plt_path}", f"Histórico + previsão salvo: {plt_path}")
plt.show()

### Modeling Output

This final step provides a definitive audit of all artifacts generated by the project. It systematically lists the contents of the output directories (processed_data and figures).

It serves three main purposes:

- Verification: Confirms that all expected outputs (evaluation tables, model files, forecasts, and visualizations) were successfully created and saved.

- Organization & Reproducibility: Provides a comprehensive summary of the project's key assets, essential for anyone consuming or reproducing the results.

- Quick Location: Helps locate critical files, such as the serialized SARIMA model (.pkl) and the final 2011 forecasts (.csv).

In [ ]:
# EN: Save key outputs and list files
# PT-BR: Salva saídas chave e lista os arquivos

# Prints the list of saved output files in the processed_data directory.
print("Saved files in processed_data:")
# Uses glob to find and list all CSV files, sorted alphabetically.
for p in sorted(DATA_DIR.glob("*.csv")):
    print("-", p.name)
# Finds and lists all model files saved in pickle format (.pkl).
for p in sorted(DATA_DIR.glob("*.pkl")):
    print("-", p.name)

# Prints the list of generated figures.
print("\nFigures in figures/:")
# Finds and lists all saved PNG image files.
for p in sorted(FIG_DIR.glob("*.png")):
    print("-", p.name)